In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

def detect_wire_strands(image_path):
    # Loading the image
    image = cv2.imread(image_path)

    # Converting the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Applying Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Applying adaptive thresholding to obtain binary image
    _, threshold = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    # Finding contours in the binary image
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering contours based on length and aspect ratio
    min_length = 20  # Minimum length of a strand
    max_aspect_ratio = 2  # Maximum aspect ratio of a strand

    wire_strands = []
    for contour in contours:
        # Calculating the length of the contour
        length = cv2.arcLength(contour, True)

        # Filtering contours based on length and aspect ratio
        if length > min_length:
            (x, y, w, h) = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h

            if aspect_ratio < max_aspect_ratio:
                wire_strands.append(contour)

    # Refining the wire strands by removing overlaps and small contours
    refined_strands = refine_wire_strands(image, wire_strands)

    # Drawing contours around the detected wire strands
    image_with_strands = cv2.drawContours(image.copy(), refined_strands, -1, (0, 255, 0), 2)

    # Displaying the image with wire strands
    cv2_imshow(image_with_strands)

    # Print the count of detected wire strands
    count = len(refined_strands)
    print(f'Wire Strand Count: {count}')

def refine_wire_strands(image, contours):
    # Creating a binary mask for wire strands
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

    # Performing morphological operations to improve strand separation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opened = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Finding contours in the opened mask
    refined_contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering the contours based on area and aspect ratio
    min_area = 100  # Minimum area of a wire strand
    max_aspect_ratio = 2  # Maximum aspect ratio of a wire strand
    filtered_contours = []
    for contour in refined_contours:
        area = cv2.contourArea(contour)
        (x, y, w, h) = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h

        if area > min_area and aspect_ratio < max_aspect_ratio:
            filtered_contours.append(contour)

    return filtered_contours


# Call the function to detect wire strands in an image
image_path = '/content/images (2).jpeg'
detect_wire_strands(image_path)
